### 데이터 처리





In [ ]:
# Google Colab을 사용하는 경우 해당 셀을 실행하세요
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pandas-profiling==2.*
    !pip install pdpbox
    !pip install shap

In [ ]:
#드라이브에 접근할 수 있도록 아래 코드 입력
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Importing Models
from sklearn.ensemble import RandomForestClassifier #,BaggingRegressor,AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.compose import TransformedTargetRegressor

#importing Preprocess 
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from category_encoders import OrdinalEncoder, TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from scipy.stats import randint,uniform

#importing Evaluation Metrics
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score, f1_score
from sklearn.metrics import mean_squared_log_error
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
#불러올 파일의 경로를 filename 변수에 저장
filename = '/content/drive/MyDrive/AirPollutionSeoul/Measurement_summary.csv'
filename2 = '/content/drive/MyDrive/AirPollutionSeoul/Original Data/Measurement_item_info.csv'

In [ ]:
#pandas read_csv로 불러오기

df = pd.read_csv(filename)
item_info = pd.read_csv(filename2)
df.head()

In [ ]:
df.tail()

In [ ]:
item_info # Standard value of all of the factors

### EDA


In [ ]:
#data cleaning

df.isnull()

In [ ]:
# checking weather data has any missing value or not
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

Now we can conclude there is no any missing value in data given as there is plane bar for all factors

In [ ]:
df.info()

In [ ]:
req_data=df[['SO2','NO2','O3','CO','PM10','PM2.5']]
req_data.describe()

conclusion from description of data-

SO2 has mean count '-0.001795' which is much lesser than Good satandard.

NO2 has mean count '0.022519' which is also lesser than Good satandard of NO2.

O3 has mean count '0.017979' which is also lesser than Good satandard of O3.

CO has mean count '0.017979' which is also lesser than Good satandard of CO.

PM10 has mean count '43.708051' it will come under Normal standard of PM10.

PM2.5 has mean count '25.411995' it will also come under Normal standard of PM2.5.

so overall Seoul has good air Quality

In [ ]:
#그래서 무슨 의민데?
plt.figure(figsize=(12,8))
sns.boxplot(data=req_data)

### 컬럼추가


In [ ]:
# converting 'SO2' value in their standard
def measure(x):
    if x<=0.02:
        return 'good'
    elif 0.02<x<=0.05:
        return 'normal'
    elif 0.05<x<=0.15:
        return 'bad'
    else:
        return 'very bad'
    
SO2_measure=list(map(measure,req_data['SO2']))

In [ ]:
# converting 'NO2' value in their standard
def measure(x):
    if x<=0.03:
        return 'good'
    elif 0.03<x<=0.06:
        return 'normal'
    elif 0.06<x<=0.20:
        return 'bad'
    else:
        return 'very bad'
    
NO2_measure=list(map(measure,req_data['NO2']))

In [ ]:
# converting 'O3' value in their standard
def measure(x):
    if x<=0.02:
        return 'good'
    elif 0.02<x<=0.05:
        return 'normal'
    elif 0.05<x<=0.15:
        return 'bad'
    else:
        return 'very bad'
    
O3_measure=list(map(measure,req_data['O3']))

In [ ]:
# converting 'CO' value in their standard
def measure(x):
    if x<=2.00:
        return 'good'
    elif 2.00<x<=9.00:
        return 'normal'
    elif 9.00<x<=15.00:
        return 'bad'
    else:
        return 'very bad'
    
CO_measure=list(map(measure,req_data['CO']))

In [ ]:
# converting 'PM10' value in their standard
def measure(x):
    if x<=30.00:
        return 'good'
    elif 30.00<x<=80.00:
        return 'normal'
    elif 80.00<x<=150.00:
        return 'bad'
    else:
        return 'very bad'
    
PM10_measure=list(map(measure,req_data['PM10']))

In [ ]:
# converting 'PM2.5' value in their standard
def measure(x):
    if x<=15.00:
        return 'good'
    elif 15.00<x<=35.00:
        return 'normal'
    elif 35.00<x<=75.00:
        return 'bad'
    else:
        return 'very bad'
    
PM25_measure=list(map(measure,req_data['PM2.5']))

In [ ]:
#'O3' 주의보 발령은 '1', 주의보 발령 아닐 경우 '0'
def measure2(x):
    if x>=0.12:
        return '1'
    else:
        return '0'
    
O3_measure2=list(map(measure2,req_data['O3']))

In [ ]:
#'PM10' 주의보 발령은 '1', 주의보 발령 아닐 경우 '0'
def measure2(x):
    if x>=150:
        return '1'
    else:
        return '0'
    
PM10_measure2=list(map(measure2,req_data['PM10']))

In [ ]:
#'PM2.5' 주의보 발령은 '1', 주의보 발령 아닐 경우 '0'
def measure2(x):
    if x>=75:
        return '1'
    else:
        return '0'
    
PM25_measure2=list(map(measure2,req_data['PM2.5']))

In [ ]:
lst = []
for i in req_data['SO2']:
    if i > 0.05:
        lst.append(1)
    else:
        lst.append(0)

test_df = pd.DataFrame(lst)

In [ ]:
lst = []
for i in req_data['NO2']:
    if i > 0.06:
        lst.append(1)
    else:
        lst.append(0)

test_df2 = pd.DataFrame(lst)

In [ ]:
lst = []
for i in req_data['O3']:
    if i > 0.05:
        lst.append(1)
    else:
        lst.append(0)

test_df3 = pd.DataFrame(lst)

In [ ]:
lst = []
for i in req_data['CO']:
    if i > 9:
        lst.append(1)
    else:
        lst.append(0)

test_df4 = pd.DataFrame(lst)

In [ ]:
lst = []
for i in req_data['PM10']:
    if i >80:
        lst.append(1)
    else:
        lst.append(0)

test_df5 = pd.DataFrame(lst)

In [ ]:
lst = []
for i in req_data['PM2.5']:
    if i >35:
        lst.append(1)
    else:
        lst.append(0)

test_df6 = pd.DataFrame(lst)

In [ ]:
req_data['SO2_standard']=SO2_measure
req_data['NO2_standard']=NO2_measure
req_data['O3_standard']=O3_measure
req_data['CO_standard']=CO_measure
req_data['PM10_standard']=PM10_measure
req_data['PM2.5_standard']=PM25_measure
req_data['O3_warning']=O3_measure2
req_data['PM10_warning']=PM10_measure2
req_data['PM2.5_warning']=PM25_measure2
req_data['SO2_bad'] = test_df
req_data['NO2_bad'] = test_df2
req_data['O3_bad'] = test_df3
req_data['CO_bad'] = test_df4
req_data['PM10_bad'] = test_df5
req_data['PM2.5_bad'] = test_df6

### target 만들기


In [ ]:
req_data['total_bad'] = req_data['SO2_bad'] + req_data['NO2_bad'] + req_data['O3_bad'] + req_data['CO_bad'] + req_data['PM10_bad'] + req_data['PM2.5_bad']
req_data['total_bad']

In [ ]:
lst =[]
for i in req_data['total_bad']:
    if i == 4:
        lst.append(1)

    else:
        lst.append(0)

In [ ]:
req_data['total_bad'] = pd.DataFrame(lst)

In [ ]:
req_data['total_bad']

In [ ]:
req_data

### 타겟 확인


In [ ]:
sum(req_data['total_bad']==1)

In [ ]:
sum(req_data['PM2.5_warning']==1)

In [ ]:
sum(req_data['PM10_warning']==1)

In [ ]:
#so2의 품질기준 분석
sns.countplot(x='SO2_standard',data=req_data)

In [ ]:
#no2의 품질기준 분석
sns.countplot(x='NO2_standard',data=req_data)

In [ ]:
#o3의 품질기준 분석
sns.countplot(x='O3_standard',data=req_data)

In [ ]:
#co의 품질기준 분석
sns.countplot(x='CO_standard',data=req_data)

In [ ]:
#PM10의 품질기준 분석
sns.countplot(x='PM10_standard',data=req_data)

In [ ]:
#PM2.5의 품질기준 분석
sns.countplot(x='PM2.5_standard',data=req_data)

From above figures we can see that SO2 and CO level is quite good as all the data observed to be of 'good' standard. O3 has much lesser quality than these two factors as some data are observed of 'normal' and 'bad' standard and PM10 and PM2.5 has to be notice as there is very_bad standard of these factors observed.

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(req_data.corr(),
            vmin=0,
            cmap='coolwarm',
            annot=True);

At here from the above picture we can see that there is high correlation in between 'SO2 and O3' and lowest correlation is in between'O3 and PM 2.5'

In [ ]:
req_data.to_csv('/content/drive/MyDrive/req_data.csv', index = False)

In [ ]:
data = req_data.drop(columns=['SO2_bad', 'NO2_bad', 'CO_bad', 'O3_bad', 'PM10_bad', 'PM2.5_bad'])
data.info()

### 특성엔지니어링

In [ ]:
#특성엔지니어링

def engineer(data):
    data['SO2_standard'] = pd.to_numeric(data['SO2_standard'].replace(['good', 'normal', 'bad', 'very bad'], [0,1,2,3]))

    data['NO2_standard'] = pd.to_numeric(data['NO2_standard'].replace(['good', 'normal', 'bad', 'very bad'], [0,1,2,3]))

    data['CO_standard'] = pd.to_numeric(data['CO_standard'].replace(['good', 'normal', 'bad', 'very bad'], [0,1,2,3]))
    
    data['O3_standard'] = pd.to_numeric(data['O3_standard'].replace(['good', 'normal', 'bad', 'very bad'], [0,1,2,3]))
 
    data['PM2.5_standard'] = pd.to_numeric(data['PM2.5_standard'].replace(['good', 'normal', 'bad', 'very bad'], [0,1,2,3]))

    data['PM10_standard'] = pd.to_numeric(data['PM10_standard'].replace(['good', 'normal', 'bad', 'very bad'], [0,1,2,3]))

    return data

In [ ]:
data = engineer(data)

In [ ]:
data1 = data[data['total_bad']==1]
data1

In [ ]:
data = data[data['total_bad']==0]
data2 = data.sample(n=10000)
data2

In [ ]:
data3 = data.sample(n=300)
data3

In [ ]:
dataset = pd.concat([data1,data2], ignore_index=True)
dataset

In [ ]:
dataset = dataset.drop(columns=['SO2_standard',	'NO2_standard',	'O3_standard', 'CO_standard', 'PM10_standard',	'PM2.5_standard', 'O3_warning',	'PM10_warning',	'PM2.5_warning'])
dataset

In [ ]:
dataset2 = pd.concat([data1, data3], ignore_index=True)
dataset2

In [ ]:
dataset2 = dataset2.drop(columns=['SO2_standard', 'NO2_standard', 'O3_standard', 'CO_standard', 'PM10_standard', 'PM2.5_standard', 'O3_warning', 'PM10_warning', 'PM2.5_warning'])
dataset2

In [ ]:
dataset.to_csv('/content/drive/MyDrive/Airpollution_dataset.csv', index = False)
dataset2.to_csv('/content/drive/MyDrive/Airpollution_dataset2.csv', index = False)

In [ ]:
#훈련/검증세트로 나누기
target = 'total_bad'


train, test = train_test_split(dataset, train_size=0.8, 
                               stratify=dataset[target], random_state=2)

train.shape, test.shape

In [ ]:
train, val = train_test_split(train, train_size = 0.8,
                              stratify=train[target], random_state=2)

train.shape, val.shape, test.shape

In [ ]:
train2, test2 = train_test_split(dataset2, train_size=0.8, 
                               stratify=dataset2[target], random_state=2)

train2.shape, test2.shape

In [ ]:
train2, val2 = train_test_split(train2, train_size = 0.8,
                              stratify=train2[target], random_state=2)

train2.shape, val2.shape, test2.shape

In [ ]:
# train2 = engineer(train2)
# val = engineer(val)
# test = engineer(test)

In [ ]:
features = train.drop(columns=[target]).columns

In [ ]:
#훈련/검증/데이터 데스트 특성과 타겟으로 분리
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [ ]:
X_train2 = train2[features]
y_train2 = train2[target]
X_val2 = val2[features]
y_val2 = val2[target]
X_test2 = test2[features]

In [ ]:
pipe = make_pipeline(
    SimpleImputer(),
    RandomForestClassifier(max_depth= 6, #9
                           #min_samples_split = 10,
                           #min_samples_leaf = 20, 
                           #max_leaf_nodes=10, 
                           #max_features = 24, 
                           n_estimators = 200, #100
                            class_weight='balanced',
                           n_jobs=-1,  
                           random_state=2, 
                           oob_score=True))

pipe.fit(X_train, y_train)
  
y_val_pred = pipe.predict(X_val)

print('훈련 정확도: ', pipe.score(X_train, y_train))
print('검증 정확도: ', pipe.score(X_val, y_val))
print('f1_Score', f1_score(y_val, y_val_pred))

In [ ]:
pipe.fit(X_train2, y_train2)

y_val_pred2 = pipe.predict(X_val2)

print('훈련 정확도: ', pipe.score(X_train2, y_train2))
print('검증 정확도: ', pipe.score(X_val2, y_val2))
print('f1_Score', f1_score(y_val2, y_val_pred2))

In [ ]:
pipe.predict([[0.06, 0.07, 0.06, 10, 81, 36]])
